# Program 10 (Lab-10)

Write a program to Build a Recommendation system using Deep Learning techniques.

In [1]:
import pandas as pd
import requests
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense
from tensorflow.keras.optimizers import Adam

In [2]:
data = pd.read_csv('ratings.csv')

In [3]:
data.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
user_ids = data.userId.unique()
item_ids = data.movieId.unique()
user2idx = {u: i for i,u in enumerate(user_ids)}
item2idx = {m: i for i,m in enumerate(item_ids)}

In [5]:
num_users = len(user_ids)
num_items = len(item_ids)

In [6]:
train_data, val_data = train_test_split(data,test_size=0.2,random_state=42)

In [7]:
user_input = Input(shape=(1,))
item_input = Input(shape=(1,))
user_embedding = Embedding(num_users,50)(user_input)
item_embedding = Embedding(num_items,50)(item_input)
user_vecs = Flatten()(user_embedding)
item_vecs = Flatten()(item_embedding)
dot_product = tf.keras.layers.Dot(axes=1)([user_vecs,item_vecs])
model = Model(inputs=[user_input,item_input],outputs=dot_product)

In [8]:
model.compile(optimizer=Adam(learning_rate=0.001),loss='mean_squared_error')

In [9]:
history = model.fit(
    [train_data.userId.map(user2idx),train_data.movieId.map(item2idx)],
    train_data.rating,
    batch_size=64,
    epochs=5,
    verbose=2,
    validation_data=([val_data.userId.map(user2idx), val_data.movieId.map(item2idx)],
                    val_data.rating)
)

Epoch 1/5
1261/1261 - 7s - loss: 10.4127 - val_loss: 4.0988
Epoch 2/5
1261/1261 - 5s - loss: 2.3595 - val_loss: 1.8240
Epoch 3/5
1261/1261 - 6s - loss: 1.2204 - val_loss: 1.4439
Epoch 4/5
1261/1261 - 6s - loss: 0.8937 - val_loss: 1.3212
Epoch 5/5
1261/1261 - 5s - loss: 0.7345 - val_loss: 1.2762


In [13]:
# Sample recommendation
user_id = 1
user_input = np.array([user2idx[user_id]])
item_ids = data[data.userId == user_id].sample(n=5)['movieId'].values
item_inputs = np.array([item2idx[item] for item in item_ids])
user_input = np.array([user2idx[user_id]] * len(item_ids))  # Repeat user_input
predicted_ratings = model.predict([user_input, item_inputs])

print("Sample Recommendations:")
for item_id, rating in zip(item_ids, predicted_ratings):
    print(f"Movie ID: {item_id}, Predicted Rating: {rating[0]:.2f}")


Sample Recommendations:
Movie ID: 2542, Predicted Rating: 5.14
Movie ID: 1275, Predicted Rating: 4.87
Movie ID: 151, Predicted Rating: 3.77
Movie ID: 2012, Predicted Rating: 4.52
Movie ID: 2944, Predicted Rating: 4.99
